# Delete

(This is an AI-generated placeholder to be edited.)

The `delete` command in DataJoint provides a robust mechanism for removing data from tables. It ensures that deletions respect the dependency structure defined by the relational schema, preserving the integrity of your database. This command is powerful and should be used with a clear understanding of its effects on downstream dependencies.

## Overview of `delete`

The `delete` command removes entries from a table. When executed, it ensures that all dependent data in downstream tables is also removed, unless explicitly restricted.

### Syntax

```python
<Table>.delete(safemode=True, quick=False)
```

### Parameters

1. **`safemode`** *(default: True)*:
   - If `True`, prompts the user for confirmation before deleting any data.
   - If `False`, proceeds with deletion without prompting.
2. **`quick`** *(default: False)*:
   - If `True`, accelerates deletion by skipping certain checks, such as confirming dependencies.
   - Use this option cautiously as it bypasses safety mechanisms.

## Example Usage

### Deleting Specific Entries

To delete specific rows based on a condition:

```python
import datajoint as dj

schema = dj.Schema('example_schema')

@schema
class Animal(dj.Manual):
    definition = """
    animal_id: int  # Unique identifier for the animal
    ---
    species: varchar(64)  # Species of the animal
    age: int             # Age of the animal in years
    """

# Insert example data
Animal.insert([
    {'animal_id': 1, 'species': 'Dog', 'age': 5},
    {'animal_id': 2, 'species': 'Cat', 'age': 3},
])

# Delete rows where species is 'Cat'
(Animal & {'species': 'Cat'}).delete()
```

### Deleting All Entries

To delete all entries from a table:

```python
Animal.delete()
```

### Using `safemode`

By default, `safemode=True` will prompt the user for confirmation before deletion. To bypass the prompt:

```python
Animal.delete(safemode=False)
```

## Dependency Management

One of the key features of `delete` is its handling of dependencies. When deleting data, DataJoint ensures that:

1. **Downstream Data is Removed**: Any dependent entries in other tables are recursively deleted to maintain referential integrity.
2. **Deletion is Acyclic**: The dependency graph is traversed in topological order to avoid cyclic deletion issues.

### Restricting Deletions

To delete specific entries while preserving others:

```python
(Animal & {'animal_id': 1}).delete()
```

In this example, only the entry with `animal_id=1` is deleted, and other rows remain intact.

## Best Practices

1. **Use `safemode=True`**: Always use `safemode` when testing or in uncertain situations to prevent accidental data loss.
2. **Test Deletion Queries**: Before running `delete`, test your restrictions with `fetch` to ensure you are targeting the correct data.
3. **Be Cautious with `quick=True`**: Use the `quick` parameter sparingly, as it skips important safety checks.
4. **Understand Dependencies**: Review your schema's dependency structure to anticipate the cascading effects of deletions.

## Summary

The `delete` command is a powerful tool for managing data lifecycle in a DataJoint pipeline. By respecting dependencies and offering safety mechanisms, it ensures that data deletions are controlled and consistent. Proper use of this command helps maintain the integrity and cleanliness of your database.

